Simple RAG App

Scrap the website and answer questions from that.

LLM used Gemma:2b by Ollama for Embeddings & to process information.

In [ ]:
#All imports
import os
from langchain_community.llms import Ollama
import streamlit as st 
from dotenv import load_dotenv
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser


In [5]:
load_dotenv()

#Langsmith Tracking

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [6]:
#Scrapping Site to Do Data Ingestion.

loader = WebBaseLoader(web_path=('https://siddarthvijayanand.medium.com/the-transformative-power-of-artificial-intelligence-in-political-campaigns-5404598d2858'))
docs = loader.load()
docs

[Document(metadata={'source': 'https://siddarthvijayanand.medium.com/the-transformative-power-of-artificial-intelligence-in-political-campaigns-5404598d2858', 'title': 'The Transformative Power of Artificial Intelligence in Political Campaigns | by Siddarth Vijay Anand | Medium', 'description': 'Artificial intelligence (AI) is revolutionizing the landscape of political campaigns, offering unprecedented opportunities for efficiency and effectiveness. As political parties and consultants seek…', 'language': 'en'}, page_content='The Transformative Power of Artificial Intelligence in Political Campaigns | by Siddarth Vijay Anand | MediumSitemapOpen in appSign upSign inMedium LogoWriteSign upSign inThe Transformative Power of Artificial Intelligence in Political CampaignsEmpowering Political Campaigns Through AISiddarth Vijay Anand4 min read·May 21, 2023--ListenShareAn Illustration of AI nad Political PartiesIntroductionArtificial intelligence (AI) is revolutionizing the landscape of politi

In [7]:
#Different LLMs have different context size, thus it is suggested to divide all the texts into smaller chunks.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap= 0)
split_docs = text_splitter.split_documents(docs)
split_docs

[Document(metadata={'source': 'https://siddarthvijayanand.medium.com/the-transformative-power-of-artificial-intelligence-in-political-campaigns-5404598d2858', 'title': 'The Transformative Power of Artificial Intelligence in Political Campaigns | by Siddarth Vijay Anand | Medium', 'description': 'Artificial intelligence (AI) is revolutionizing the landscape of political campaigns, offering unprecedented opportunities for efficiency and effectiveness. As political parties and consultants seek…', 'language': 'en'}, page_content='The Transformative Power of Artificial Intelligence in Political Campaigns | by Siddarth Vijay Anand'),
 Document(metadata={'source': 'https://siddarthvijayanand.medium.com/the-transformative-power-of-artificial-intelligence-in-political-campaigns-5404598d2858', 'title': 'The Transformative Power of Artificial Intelligence in Political Campaigns | by Siddarth Vijay Anand | Medium', 'description': 'Artificial intelligence (AI) is revolutionizing the landscape of po

In [8]:
#Vector Embeddings using Ollama

embeddings = OllamaEmbeddings(model="gemma:2b")
embeddocs = embeddings.embed_documents(split_docs)
embeddocs

/var/folders/s9/wm1ky0bn07l5mjx43prlt9m80000gn/T/ipykernel_36552/3855479825.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="gemma:2b")


[[-0.3186725676059723,
  -3.876551389694214,
  -1.2784080505371094,
  0.8620356917381287,
  -1.2297533750534058,
  1.300294280052185,
  1.3270776271820068,
  -0.4726357161998749,
  1.8981300592422485,
  -1.2522497177124023,
  -0.5656492114067078,
  1.3901816606521606,
  -1.4004325866699219,
  0.7491490840911865,
  0.7856063842773438,
  0.14199021458625793,
  0.3780624568462372,
  0.9708446264266968,
  0.5868393778800964,
  0.8036501407623291,
  -1.2176562547683716,
  0.11161350458860397,
  0.48384442925453186,
  0.9056099653244019,
  0.45209044218063354,
  0.544995903968811,
  0.16764280200004578,
  -0.2329367846250534,
  -0.1134849563241005,
  -0.5106805562973022,
  -0.45481550693511963,
  -0.09659646451473236,
  1.390295147895813,
  0.45973360538482666,
  0.0462348535656929,
  0.902665913105011,
  -2.3904640674591064,
  -1.2544829845428467,
  -0.26586151123046875,
  -1.5320097208023071,
  0.4192388355731964,
  -1.3658798933029175,
  -0.304070383310318,
  -0.4775640070438385,
  2.1152

In [9]:
#Storing in Vector DB using FAISS

db = FAISS.from_documents(split_docs, embeddings)
db

In [10]:
#Query from Vectorstore DB (one method.)

query = "What is AI's role in politics?"

result = db.similarity_search(query)
result[0].page_content

'considerations and responsible implementation are imperative to mitigate risks and ensure fairness.'

In [12]:
#Setting Up an LLM

llm = Ollama(model = "gemma:2b")

/var/folders/s9/wm1ky0bn07l5mjx43prlt9m80000gn/T/ipykernel_36552/2887379914.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model = "gemma:2b")


In [17]:
#Prompt
prompt = ChatPromptTemplate(
    [
        ("system", "You're an AI & Politics Expert. Please help me breakdown my questions and provide proper answers."),
        ("user","Question:{question}")
    ]
)

In [35]:
#Setting Up document chain to add context.

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    '''
    Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    Question: {input}
    '''
)

document_chain = create_stuff_documents_chain(llm,prompt)
document_chain


RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, template='\n    Answer the following question based only on the provided context:\n    <context>\n    {context}\n    </context>\n    Question: {input}\n    '), additional_kwargs={})])
| Ollama(model='gemma:2b')
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [36]:
#Setting Up Retriever of the VectorStore DB.

retriever = db.as_retriever()
retrival_chain = create_retrieval_chain(retriever, document_chain)

In [37]:
docs = retriever.get_relevant_documents("What is the role of AI in politics?")
print(docs)

[Document(id='29ca5360-4ae3-4ca8-87ba-93fe71454006', metadata={'source': 'https://siddarthvijayanand.medium.com/the-transformative-power-of-artificial-intelligence-in-political-campaigns-5404598d2858', 'title': 'The Transformative Power of Artificial Intelligence in Political Campaigns | by Siddarth Vijay Anand | Medium', 'description': 'Artificial intelligence (AI) is revolutionizing the landscape of political campaigns, offering unprecedented opportunities for efficiency and effectiveness. As political parties and consultants seek…', 'language': 'en'}, page_content='processes.In this article, we delve into the burgeoning role of AI in political campaigns,'), Document(id='707c68de-78e7-4aee-b476-745b1146385f', metadata={'source': 'https://siddarthvijayanand.medium.com/the-transformative-power-of-artificial-intelligence-in-political-campaigns-5404598d2858', 'title': 'The Transformative Power of Artificial Intelligence in Political Campaigns | by Siddarth Vijay Anand | Medium', 'descrip

In [39]:
#Getting the response from the chain

response = retrival_chain.invoke({"input": "Tell me about AI in politics in 50 words?"})

print(response['answer'])

Sure, here's a 50-word summary about AI in politics:

The context describes AI's role in enhancing political campaigns, outreach strategies, decision-making processes, and ensuring fairness in political processes.
